In [132]:
import functions_barcelona as fb
#from functions_barcelona import posant_accents,utmToLatLng,\
      #              ped_to_angles,setmana_a_angles,mes_a_angles,cause_to_angles
import datetime
import math
import pickle
import pandas as pd
import requests
import json
import os
import numpy as np
from datetime import date, datetime, timedelta
from functions_barcelona import getting_daily_weather,getting_next_day
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
import bs4
from bs4 import BeautifulSoup
import geopandas as gpd
import difflib

import webbrowser
import autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
r = requests.get('https://opendata-ajuntament.barcelona.cat/data/api/3/action/package_search?rows=1000')
response= r.json()

{'notes_translated': {'ca': 'Accidents segons causa conductor  gestionats per la Guàrdia Urbana a la ciutat de Barcelona  ',
  'en': 'Accidents by driver cause managed by the Guàrdia Urbana in the city of Barcelona',
  'es': 'Accidentes  según causa conductor gestionados por la Guàrdia Urbana a la ciutat de Barcelona'},
 'geolocation': 'Yes_with_map_view',
 'code': 'ACCIDENTS_CAUSA_CONDUCTOR_GU_BCN',
 'url_tornada': {'ca': 'http://ajuntament.barcelona.cat/guardiaurbana/ca/node',
  'en': 'http://ajuntament.barcelona.cat/guardiaurbana/en/node',
  'es': 'http://ajuntament.barcelona.cat/guardiaurbana/es/node'},
 'dataset_fields_description': "Les dades d’aquest dataset complementen les d'[Accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona](http://opendata-ajuntament.barcelona.cat/data/ca/dataset?name=accidents-gu-bcn)    \r\nEl valor '-1' en una cel·la indica que no es disposa de la dada.",
 'fuente': 'Ajuntament de Barcelona',
 'private': False,
 'num_tags': 10,
 'api': '

In [9]:
response['result']['results'][21]

{'notes_translated': {'ca': 'Relació dels accidents segons causa mediata gestionats per la Guàrdia Urbana a la ciutat de Barcelona. Un accident pot tenir una o més causes mediates\xa0les quals fan referència a factors\xa0externs del resultat en\xa0temps, lloc o grau (Ex: Alcoholèmia i Excés de velocitat o  velocitat inadequada)\xa0  \r\n \r\n',
  'en': 'List of accidents by mediate cause managed by the Guàrdia Urbana in the city of Barcelona. An accident may have one or more mediate causes that refer to external factors of the result in time, place or degree (Ex: Breathalyzer and Speeding or inappropriate speed)\r\n ',
  'es': 'Relación de los accidentes según causa mediata gestionada por la Guàrdia Urbana en la ciudad de Barcelona. Un accidente puede tener una o más causas mediatas que hacen referencia a factores externos del resultado en tiempo, lugar o grado (Ej: Alcoholemia y Exceso de velocidad o velocidad inadecuada)'},
 'geolocation': 'Yes_with_map_view',
 'code': 'ACCIDENTS_CAU

In [7]:
#response dict_keys(['help', 'success', 'result'])--str, bool, 
#response['result'] dict_keys(['count', 'sort', 'facets', 'results', 'search_facets'])----int,

for mydict in response['result']['results']:
    if 'accidents' in mydict['name']:
        print(mydict['name'].upper(),mydict['resources'][1]['format'])

ACCIDENTS_CAUSA_CONDUCTOR_GU_BCN CSV
ACCIDENTS-CAUSES-GU-BCN CSV
ACCIDENTS-TIPUS-GU-BCN CSV
ACCIDENTS-VEHICLES-GU-BCN CSV
ACCIDENTS-PERSONES-GU-BCN CSV
ACCIDENTS-GU-BCN CSV


In [36]:
for i in range(len(url)):
    if url[i]!=url2[i]:
        print(i)

In [ ]:
# for n,file in enumerate(response['result']['results']):
#     if 'accidents' in file['name']:
#         print(n)
url='https://opendata-ajuntament.barcelona.cat/data/dataset/719b1054-4166-4692-b63e-622b621b4293/resource/976d35c3-37e2-40ba-be1d-b7b5b791f68e/download'
url2='https://opendata-ajuntament.barcelona.cat/data/dataset/719b1054-4166-4692-b63e-622b621b4293/resource/976d35c3-37e2-40ba-be1d-b7b5b791f68e/download'
try:
    pd.read_csv(url2)#encoding_errors='replace')
except UnicodeDecodeError:
    #print('yes')
    pd.read_csv(url2,encoding='ISO-8859-1')#encoding_errors='replace')

In [7]:
###ORIGINAL WORKING

r = requests.get('https://opendata-ajuntament.barcelona.cat/data/api/3/action/package_search?rows=1000')
response= r.json()


def concatenating_dataframes(filter1):
    
    files=response['result']['results']
    for num, file in enumerate(files):
        
        if (('accidents-' +filter1+'gu') in file['name']) or ('accidents_' +filter1+'gu') in file['name']:
            #print(file['name'])
            filter_list=[]
            
            for fitxer in file['resources']:

                if fitxer['format']=='CSV':

                    print(fitxer['name'],fitxer['format'])
                    #print(fitxer['url'])
                    try:
                        filter_list.append(pd.read_csv(fitxer['url']))
                        #print(pd.read_csv(fitxer['url']).shape)
                    except:
                        try:
                            filter_list.append(pd.read_csv(fitxer['url'],encoding='ISO-8859-15'))
                            #print(pd.read_csv(fitxer['url']).shape)
                        except:
                            try:
                                filter_list.append(pd.read_csv(fitxer['url'],encoding="ISO-8859-1"))
                                #print(pd.read_csv(fitxer['url']).shape)
                            except :
                                filter_list.append(pd.read_csv(fitxer['url'],sep=';',encoding='ISO-8859-1'))
                    
#             column_names=filter_list[0].columns
#             data=pd.DataFrame()
#             for df in filter_list:
#                 df=pd.DataFrame.from_records(df.values)
#                 data=pd.concat([data,df])
#                 ###Make sure that the nunmber/name of final columns is the longest or filter them beforehand
#                 #data=data.append(df,ignore_index=True)
#                 #print(data.shape)
#             data.columns=column_names
#             data=data.reset_index()
    try:
        #return data, filter_list
        return filter_list
    except:
        print('NO FILE WITH THAT FILTER')
    print('CONCATENATING DONE')
#causes_df= concatenating_dataframes('causes-')

noms=['causes-', 'persones-','tipus-', 'vehicles-','','causa_conductor_']
##MAYBE ADDING CAUSA_CONDUCTOR
dict_noms={}
dict_files={}
for nom in noms:
    #dict_noms[nom[:-1]],dict_files[nom[:-1]]=concatenating_dataframes(nom)
    dict_files[nom[:-1]]=concatenating_dataframes(nom)
    print(nom +' DONE')

#pickle.dump( dict_noms, open( "../data/dataframes_dict.pkl", "wb" ) )
pickle.dump( dict_files, open( "../data/dataframes_dict_files.pkl", "wb" ) )
#favorite_color = pickle.load( open( "dataframes_dict.pkl", "rb" ) )

2022_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
2021_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
2020_ACCIDENTS_CAUSES_GU_BCN.csv CSV
2019_ACCIDENTS_CAUSES_GU_BCN.csv CSV
2018_ACCIDENTS_CAUSES_GU_BCN.csv CSV
2017_ACCIDENTS_CAUSES_GU_BCN.csv CSV
2016_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
2015_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
2014_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
2013_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
2012_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
2011_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
2010_ACCIDENTS_CAUSES_GU_BCN_.csv CSV
causes- DONE
2022_ACCIDENTS_PERSONES_GU_BCN.csv CSV
2021_ACCIDENTS_PERSONES_GU_BCN.csv CSV
2020_ACCIDENTS_PERSONES_GU_BCN.csv CSV
2019_ACCIDENTS_PERSONES_GU_BCN.csv CSV
2018_ACCIDENTS_PERSONES_GU_BCN.csv CSV
2017_ACCIDENTS_PERSONES_GU_BCN.csv CSV
2016_ACCIDENTS_PERSONES_GU_BCN_.csv CSV
2015_ACCIDENTS_PERSONES_GU_BCN_.csv CSV
2014_ACCIDENTS_PERSONES_GU_BCN_.csv CSV
2013_ACCIDENTS_PERSONES_GU_BCN_.csv CSV
2012_ACCIDENTS_PERSONES_GU_BCN_.csv CSV
2011_ACCIDENTS_PERSONES_GU_BCN_.csv CSV
2010_ACCIDENTS_PERSONES

In [28]:
repeated_nums=[]

for  num in dict_files['causes'][1]['Numero_expedient'].values:
    if num in dict_files['causa_conductor'][1]['Numero_expedient'].values:
        repeated_nums.append(num)
        if len(repeated_nums)>10:
            break
for num in repeated_nums:
    print(num,'CAUSES', dict_files['causes'][1][dict_files['causes'][1]['Numero_expedient']==num].Descripcio_causa_mediata)
    print('\n')
    print(num,'causa_conductor'.upper(),dict_files['causa_conductor'][1][dict_files['causa_conductor'][1]['Numero_expedient']==num].Descripcio_causa_mediata)

2021S007098     CAUSES 0    No hi ha causa mediata
Name: Descripcio_causa_mediata, dtype: object


2021S007098     CAUSA_CONDUCTOR 1076    Manca d'atenció a la conducció
Name: Descripcio_causa_mediata, dtype: object
2021S005729     CAUSES 1    No hi ha causa mediata
Name: Descripcio_causa_mediata, dtype: object


2021S005729     CAUSA_CONDUCTOR 6765    No determinada
Name: Descripcio_causa_mediata, dtype: object
2021S006593     CAUSES 2    No hi ha causa mediata
Name: Descripcio_causa_mediata, dtype: object


2021S006593     CAUSA_CONDUCTOR 6766    Manca d'atenció a la conducció
Name: Descripcio_causa_mediata, dtype: object
2021S004679     CAUSES 3    No hi ha causa mediata
Name: Descripcio_causa_mediata, dtype: object


2021S004679     CAUSA_CONDUCTOR 2    No respectat pas de vianants
Name: Descripcio_causa_mediata, dtype: object
2021S000960     CAUSES 4    No hi ha causa mediata
Name: Descripcio_causa_mediata, dtype: object


2021S000960     CAUSA_CONDUCTOR 6768    Manca d'atenció a 

In [15]:
dict_files=pickle.load( open( "../data/dataframes_dict_files.pkl", "rb" ) )


In [16]:
for d in dict_files['']:
    for col in d.columns:
        if 'barri' in col.lower():
            print(col)

Codi_barri
Nom_barri
Codi_barri
Nom_barri
Codi_barri
Nom_barri
Codi_barri
Nom_barri
Codi_barri
Nom_barri
Codi_barri
Nom_barri
Codi_barri
Nom_barri
Codi barri
Nom barri
NK barri
Nom barri
NK barri
Nom barri
NK barri
Nom barri
NK barri
Nom barri
NK barri
Nom barri


In [38]:
dict_files=pickle.load( open( "../data/dataframes_dict_files.pkl", "rb" ) )
columnes=['num_incident', 'district_code','district', 'neighborhood','street_name',
          'weekday', 'year', 'month', 'day', 'hour', 'ped_cause',
           'num_deaths', 'num_minorly_injured', 'num_severly_injured',
            'num_victims', 'num_vehicles', 'utm_y', 'utm_x']
replacements={"Numero_expedient": "num_incident",
              "Numero_d_expedient": "num_incident",
              "Numero_dexpedient": "num_incident",
              'N£mero_dexpedient': "num_incident",
              'Numero_Expedient': "num_incident",
              'Codi_expedient': 'num_incident',
              'Codi_dexpedient':'num_incident',
                "Nom_districte": "district",
              'Codi_districte': 'district_code',
              'Codi districte': 'district_code',
               "Nom_barri":"neighborhood",
              "Nom_carrer":"street_name",
              "Descripcio_dia_setmana":"weekday",
              "NK_Any":"year",
              "Any":"year",
              "Nom_mes":"month",
              "Dia_mes":"day",
              "Dia_de_mes":"day",
              "Hora_dia":"hour",
              "Descripcio_causa_vianant":"ped_cause",
              "Numero_morts":"num_deaths",
              "Numero_lesionats_lleus":"num_minorly_injured",
              "Numero_lesionats_greus":"num_severly_injured",
              "Numero_victimes":"num_victims",
              "Numero_vehicles_implicats":"num_vehicles",
              "Coordenada_UTM_X_ED50":"utm_x",
              "Coordenada_UTM_Y_ED50":"utm_y",
            "Coordenada_UTM_X":"utm_x",
             "Coordenada_UTM_Y":"utm_y",
             "Coordenada UTM (Y)": "utm_y",
             "Coordenada UTM (X)": "utm_x",
             "Coordenada_UTM_(Y)":"utm_y",
             "Coordenada_UTM_(X)":"utm_x",
             "Longitud_WGS84":"Longitud",
             "Latitud_WGS84":"Latitud",
             'Num_postal_':"Num_postal",
             "Num_postal_caption":"Num_postal",
             'Tipus_accident':'accident_type',
              'Descripcio_tipus_accident':'accident_type',
              'Descripcio_color':'vehicle_color',
              'Descripcio_model':'vehicle_model',
              'Descripcio_marca':'vehicle_brand_name',
              'Descripcio_carnet': 'license',
              'Antiguitat_carnet':'license_seniority'
              
             }

llista_=['causes', 'persones', 'tipus', 'vehicles', '']
for grup in llista_:
    for num, df in enumerate(dict_files[grup]):
        df.columns=[col.replace("ú",'u').replace("ó","o").replace("'",'').replace(' ','_').replace("_de_","_").replace("í","i").replace("¢",'o') for col in df]

        df=df.rename(columns=replacements,)
        dict_files[grup][num]=df
acc=pd.DataFrame(columns=columnes)
    
for dd in dict_files['']:
    columns=[col for col in dd.columns if col in columnes]
    dd_final=dd[columns]
    acc=pd.concat([acc,dd_final])
    #print(acc.shape)


##Fixing utm_x and utm_y that are mixed in some cases. Replacing nulls(-1) with the mean
acc['utm_x']=[x[0] if len(str(x[0]).split('.')[0])==7 else x[1] for x in zip(acc.utm_x,acc.utm_y)]
acc['utm_y']=[x[1] if len(str(x[1]).split('.')[0])==6 else x[0] for x in zip(acc.utm_x,acc.utm_y)]

##Fixing other stuff
acc['ped_cause']=acc['ped_cause'].apply(fb.ped_to_angles)
acc['ped_cause']=acc['ped_cause'].apply(fb.ped_to_angles)
acc['weekday']=acc['weekday'].apply(fb.setmana_a_angles)
acc['month']=acc['month'].apply(fb.mes_a_angles)
acc['num_incident']=[x.strip() for x in acc['num_incident']]
#print(acc.shape)
acc=acc.drop_duplicates(subset='num_incident',keep='last')
print(acc.shape)


dups=acc[acc['num_incident'].duplicated(keep=False)]
dups[dups[['num_incident','ped_cause']].duplicated()]
##all duplicates are due to have 2 different ped cause being the rest exactly the same. Total dups: 27. I will just keep the first. No big deal; does not pay wasting time
acc=acc.dropna()
acc['district_code']=[str(int(x)) if x <0 or x>9 else '0'+str(int(x)) for x in acc.district_code]
acc.to_csv('../data/accidents_only2022.csv',index=False)
#print(acc.isnull().sum())
print("DONE WITH ACCIDENTS")


#CAUSES

for num, df in enumerate(dict_files['causes']):
    columns_to_save=[]
    for col in df:
        if ('incident' in col.lower()) or ('mediata' in col.lower()):
            columns_to_save.append(col)
    
    dict_files['causes'][num]=df[columns_to_save]
    dict_files['causes'][num].columns=['num_incident' if 'incident' in col else 'cause' for col in columns_to_save]
    
causes=pd.concat(dict_files['causes']).reset_index(drop=True)
causes['num_incident']=[x.strip() for x in causes['num_incident']]
causes['cause']=causes.cause.apply(fb.posant_accents).apply(fb.cause_to_angles)
causes=causes.drop_duplicates('num_incident',keep='last')
causes.to_csv('../data/causes2022.csv')
print(acc.shape,causes.shape)
total= pd.merge(acc,causes, how='inner',on='num_incident')
print('TOTAL',total.shape)
print("DONE WITH CAUSES")

#descripcio situacio: no interessa, no te valor predictiu
# No intereest either on 'Descripcio_victimitzacio','Descripcio_Motiu_desplaçament_vianant','Descripcio_Motiu_desplaçament_conductor'
chosen_features=['num_incident','Descripcio_sexe','Edat','Descripcio_tipus_persona','Descripcio_tipus_persona','Desc_Tipus_vehicle_implicat']

mapping_columns={'num_incident':'num_incident',
             'Descripcio_sexe':'gender',
             'Edat':'age',
            'Descripcio_tipus_persona':'people_role',
                'Desc_Tipus_vehicle_implicat': 'vehicle',
                'Desc._Tipus_vehicle_implicat': 'vehicle',}
persones=[]

for d in dict_files['persones']:
    #print(d.columns)
    d.columns=[fb.posant_accents(col) for col in d.columns]
    d.columns=[col if not col.startswith('Desc._') else 'Desc_Tipus_vehicle_implicat' for col in d.columns]
    n_columns=[col for col in d.columns if col in chosen_features]
    f=d[n_columns].copy()
    f.columns=[mapping_columns[columna] for columna in f.columns]
    persones.append(f)
    
persones=pd.concat(persones)
persones=persones.reset_index()
persones['num_incident']=[x.strip() for x in persones.num_incident]
persones=persones.replace('Dona',1).replace('Home',0).replace("Conductor",'driver').replace('Vianant', 'pedestrian').replace("Passatger",'passenger').replace('Desconegut',np.NaN).replace(-1,np.NaN)
persones['age']=persones['age'].astype(float)
persones['vehicle']=persones.vehicle.map(fb.map_vehicles)
for ix in persones[(persones.people_role=='driver')&(persones.age==0)].index:
    persones.loc[ix,'age']=np.NaN
for ix in persones[(persones.age>90)&(persones.people_role=='driver')].index:
    persones.loc[ix,'age']=np.NaN
persones['vehicle']=persones.vehicle.fillna('unknown')
persones['vehicle']=['truck' if 'truck' in x else 'bus' if ' bus' in x or x=='tram' else 'car' if x=='suv' else 'van' if 'minibus' in x else x for x in persones.vehicle]
persones['vehicle']=persones.vehicle.replace('unkown',np.NaN)

##TODO: once i fix the age issue based in the vehicle
misc_vehicles=persones.vehicle.value_counts()[persones.vehicle.value_counts()<1000].index
persones['vehicle']=[veh if veh not in misc_vehicles else 'misc_vehicle' for veh in persones.vehicle]

people=persones.set_index('num_incident').drop('index',axis=1)

people=pd.get_dummies(people)

index_1=people.index
vehicle_cols=[col for col in people.columns if 'vehicle_' in col]


scaler = MinMaxScaler()
people = pd.DataFrame(scaler.fit_transform(people), columns = people.columns)


imputer = KNNImputer(n_neighbors=5)
people = pd.DataFrame(imputer.fit_transform(people),columns = people.columns)

people=pd.DataFrame(scaler.inverse_transform(people), columns = people.columns)
vehicle_df=people[vehicle_cols].copy()
#people=people.drop(vehicle_cols,axis=1)
#people.isnull().sum()
people['gender_100/1']=[100 if gen<=0.5 else 1 for gen in people.gender]
people['age_driver']=people.age*people.people_role_driver
people['gender_driver']=people['gender_100/1']*people.people_role_driver
if 'gender' in list(people.columns):
    people=people.drop('gender',axis=1)
people=people.set_index(index_1)
people['driver_u_25']=[1 if x < 27 else 0 for x in people.age_driver]
people=people.reset_index().groupby('num_incident').sum()
people['age_driver']=[x[0]/x[1] if (x[1]!=0) else np.nan for x in zip(people['age_driver'],people['people_role_driver'])]
people['gender_driver_male']=[int(str(gen)[0]) if len(str(gen))==3 else 0 for gen in people.gender_driver.astype(int)]
people['gender_driver_female']=[int(str(gen)[2]) if len(str(gen))==3 else gen for gen in people.gender_driver.astype(int)]
people=people.drop(['age','gender_100/1','gender_driver'],axis=1)
#people.merge()

total= pd.merge(total,people, how='left',on='num_incident')
total['age_driver']=total.age_driver.fillna(total.age_driver.mean())
#print(total.isnull().sum())
people.to_csv('../data/people2022.csv')

### NEXT STEP: What to do with the nulls in total after merging with people?WILL WAIT UNTIL FINAL TOTAL

print("DONE WITH PEOPLE")

###TYPE

##TYPE

tipus=pd.DataFrame(columns=['num_incident','accident_type'])

for df in dict_files['tipus'].copy():
    
    tipus=pd.concat([tipus,df[['num_incident','accident_type']]])
    



type_accident_map={'Atropellament': 'run_over',
         'Col.lisio lateral': 'lateral_collision',
        'Xoc contra element estatic': 'crash_into_stationary',
      'Abast': 'rear-end_collision',
       'Col.lisio frontal':'frontal_collision',
      'Col.lisio fronto-lateral':'frontal-lateral_collision',
      'Caiguda (dues rodes)':'fall--motorcycle',
      'Abast multiple':'multiple_rear-end_collision',
      'Caiguda interior vehicle':'fall_inside_vehicle',
      'Altres':'Other_types',
      'Bolcada (mes de dues rodes)':'overturning',
      'Desconegut':'unknown',
      'Sortida de via amb xoc o col.lisio':'run-off_with_crash_or_collision',
      'Encalç':'rear-end_collision',
      'Sortida de via amb bolcada':'run-off_with_overturning',
      'Xoc amb animal a la calçada':'crash_into_animal_on_road',
      'Resta sortides de via':'run-off_not_included_previously'}
tipus.shape

tipus['accident_type']=tipus['accident_type'].apply(fb.posant_accents).map(type_accident_map)

tipus['num_incident']=[x.strip() for x in tipus.num_incident]
tipus=tipus.dropna()
misc_type=list(tipus.accident_type.value_counts()[tipus.accident_type.value_counts()<350].index)
misc_type.append('Other_types')
tipus=tipus.groupby('num_incident')['accident_type'].agg(lambda x: ','.join(x)).reset_index()
tipus['accident_type']=["misc_type" if x in misc_type else 'frontal' if 'frontal' in x else 'collision' if 'collision' in x else 'crash' if 'crash' in x else x for x in tipus.accident_type]


type_list=tipus.accident_type.value_counts().index
tipus['accident_type']=[fb.organizing_types(x,type_list) for x in  tipus.accident_type]
tipus2=tipus.copy()
total4=total.copy()
total= pd.merge(total,tipus, how='left',on='num_incident')
#total.fillna('misc_type',inplace=True)
total5=total.copy()
tipus.to_csv('../data/types2022.csv')

print('type: ', tipus.shape, 'Total: ',total.shape)

print("DONE WITH TYPE")

##VEHICLE***Model no surt a tots els anys

columns_to_add=['num_incident', 'vehicle_model', 'vehicle_brand_name',
       'vehicle_color', 'license', 'license_seniority' ]
vehicles=pd.DataFrame(columns=columns_to_add)

for df in dict_files['vehicles'].copy():
    
    vehicles=pd.concat([vehicles,df[columns_to_add]])
    
vehicles=vehicles.dropna()
vehicles['num_incident']=[x.strip() for x in vehicles.num_incident]
vehicles2=vehicles.copy()
vehicles=vehicles.groupby('num_incident').agg(lambda x: ','.join(x)).reset_index()
vehicles.to_csv('../data/vehicles2022.csv')
total= pd.merge(total,vehicles, how='left',on='num_incident')
##I will imput the most often
total=total.apply(lambda x: x.fillna(x.value_counts().index[0]))

total.to_csv('../data/accidents2022.csv')
#total.fillna(-1,inplace=True)
print('vehicles: ', vehicles.shape, 'Total: ',total.shape)
print('DONE')



### NEXT STEP: What to do with the nulls in total after merging with people?WILL WAIT UNTIL FINAL TOTAL
url = "https://www.youtube.com/watch?v=Udt-9J8nzGE"
webbrowser.open(url,new=1)


(118617, 18)
DONE WITH ACCIDENTS
(118614, 18) (118596, 2)
TOTAL (118593, 19)
DONE WITH CAUSES
DONE WITH PEOPLE
type:  (118599, 2) Total:  (118593, 37)
DONE WITH TYPE
vehicles:  (118524, 6) Total:  (118593, 42)
DONE


True

In [133]:
acc=pd.read_csv('../data/accidents_only2022.csv')
outfilepath="/Users/fcbnyc/mystuff/repos/BarcelonaAccidents/FINAL_FILES/tableau/shapefiles_barrio_barcelona.shp"
gdf = gpd.read_file(outfilepath)
gdf['names']= gdf['n_barri']
#gdf['n_barri'] = 
gdf['n_barri']=gdf['n_barri'].apply(lambda x: difflib.get_close_matches(x, ll['neighborhood'])[0])
gdf.to_file(outfilepath,index=False)

In [131]:

gdf['names']= gdf['n_barri']
#gdf['n_barri'] = 
gdf['n_barri']=gdf['n_barri'].apply(lambda x: difflib.get_close_matches(x, ll['neighborhood'])[0])
gdf.to_file(outfilepath)


0            Vallvidrera, el Tibidabo i les Planes
1                                           Sarrià
2                       Sant Gervasi - la Bonanova
3                                            Horta
4                       la Marina del Prat Vermell
                          ...                     
68                                   el Bon Pastor
69                             la Verneda i la Pau
70                                     Sant Andreu
71    Diagonal Mar i el Front Marítim del Poblenou
72                           el Besòs i el Maresme
Name: n_barri, Length: 73, dtype: object

In [127]:
[unidecode(barri) for barri in gdf['n_barri']]

['Vallvidrera, el Tibidabo i les Planes',
 'SarriAA ',
 'Sant Gervasi - la Bonanova',
 'Horta',
 'la Marina del Prat Vermell',
 'el Raval',
 'BarAA3 de Viver',
 'Can BarAA3',
 'Montbau',
 'la Sagrada FamAAlia',
 'el Poble Sec',
 "el Camp d'en Grassot i GrAA cia Nova",
 'el Carmel',
 'Sant GenAAs dels Agudells',
 'les Tres Torres',
 'el Putxet i el FarrAA3',
 'la Vila de GrAA cia',
 'Verdun',
 "l'Antiga Esquerra de l'Eixample",
 'la Trinitat Nova',
 "la Nova Esquerra de l'Eixample",
 'Vallbona',
 "el Camp de l'Arpa del Clot",
 'el Barri GAA2tic',
 "la Dreta de l'Eixample",
 'Sant Antoni',
 'la Barceloneta',
 'Sant Pere, Santa Caterina i la Ribera',
 'el Fort Pienc',
 'Sants - Badal',
 'la Font de la Guatlla',
 'Sant MartAA de ProvenAASSals',
 'Hostafrancs',
 'la Bordeta',
 'la Marina de Port',
 'Sants',
 'el Baix GuinardAA3',
 'les Corts',
 'la Maternitat i Sant Ramon',
 'Sant Gervasi - Galvany',
 'el Clot',
 'Porta',
 'Navas',
 'la Vila OlAAmpica del Poblenou',
 'Pedralbes',
 'Vallcarc

In [125]:
from unidecode import unidecode
accidents_barris=[]
set([unidecode(barri) for barri in ll.neighborhood]) 

{'Baro de Viver',
 'Can Baro',
 'Can Peguera',
 'Canyelles',
 'Ciutat Meridiana',
 'Desconegut',
 'Diagonal Mar i el Front Maritim del Poblenou',
 'Horta',
 'Hostafrancs',
 'Montbau',
 'Navas',
 'Pedralbes',
 'Porta',
 'Provencals del Poblenou',
 'Sant Andreu',
 'Sant Antoni',
 'Sant Genis dels Agudells',
 'Sant Gervasi - Galvany',
 'Sant Gervasi - la Bonanova',
 'Sant Marti de Provencals',
 'Sant Pere, Santa Caterina i la Ribera',
 'Sants',
 'Sants - Badal',
 'Sarria',
 'Torre Baro',
 'Vallbona',
 'Vallcarca i els Penitents',
 'Vallvidrera, el Tibidabo i les Planes',
 'Verdun',
 'Vilapicina i la Torre Llobeta',
 'el Baix Guinardo',
 'el Barri Gotic',
 'el Besos i el Maresme',
 'el Bon Pastor',
 "el Camp d'en Grassot i Gracia Nova",
 "el Camp de l'Arpa del Clot",
 'el Carmel',
 'el Clot',
 'el Coll',
 'el Congres i els Indians',
 'el Fort Pienc',
 'el Guinardo',
 'el Parc i la Llacuna del Poblenou',
 'el Poble Sec',
 'el Poble-sec',
 'el Poblenou',
 'el Putxet i el Farro',
 'el Raval',

In [81]:
import geopandas as gpd
gdf = gpd.read_file("/Users/fcbnyc/mystuff/repos/BarcelonaAccidents/FINAL_FILES/tableau/shapefiles_barrio_barcelona.shp",encoding="ISO-8859-15")
gdf[['c_barri','n_barri']].shape

(73, 2)

In [134]:
acc

,num_incident,district_code,district,neighborhood,street_name,weekday,year,month,day,hour,ped_cause,num_deaths,num_minorly_injured,num_severly_injured,num_victims,num_vehicles,utm_y,utm_x
0,2022S007749,-1,Desconegut,Desconegut,Fernando Pessoa ...,Monday,2022.0,December,19.0,11.0,No peds fault,0.0,1.0,0.0,1.0,2.0,432684.59,4588039.29
1,2022S003422,-1,Desconegut,Desconegut,Bac de Roda / Ramon Turró ...,Saturday,2022.0,June,4.0,14.0,No peds fault,0.0,2.0,0.0,2.0,2.0,433936.91,4584121.59
2,2022S003346,-1,Desconegut,Desconegut,Viladrosa ...,Wednesday,2022.0,June,1.0,21.0,No peds fault,0.0,1.0,0.0,1.0,2.0,431308.23,4588533.51
3,2022S006073,-1,Desconegut,Desconegut,A Zona Franca / Número 3 Zona Franca ...,Saturday,2022.0,October,8.0,21.0,No peds fault,0.0,0.0,0.0,0.0,1.0,428020.43,4576283.26
4,2022S005123,-1,Desconegut,Desconegut,Carles Pi i Sunyer ...,Friday,2022.0,August,26.0,16.0,No peds fault,0.0,1.0,0.0,1.0,2.0,431029.88,4582053.59
